In [5]:
import json
from dateutil.parser import parse
import pprint

In [6]:
f = open('../data/fhir/Abraham100_Klein929_8ff56a9d-b91c-48c7-a6e9-a55344881a36.json', 'r')
text = f.read()
f.close()
print(type(text))

<class 'str'>


In [7]:
with open('../data/fhir/Abraham100_Klein929_8ff56a9d-b91c-48c7-a6e9-a55344881a36.json') as f:
    bundle = json.load(f)
print(type(bundle))

<class 'dict'>


In [8]:
for entry in bundle['entry']:
    resource = entry['resource']
    resource_type = resource['resourceType']
    if resource_type == 'Claim':
        id = resource['id']
        patient = resource['patient']['reference']
        status = resource['status']
        total = resource['total']['value']
        currency = resource['total']['currency']
        start = resource['billablePeriod']['start']
        end = resource['billablePeriod']['end']
        print(patient, status, total, currency, start, end)
        break

urn:uuid:07b3c863-ea51-45a7-b446-493b39eb72e2 active 125.0 USD 1992-07-31T05:18:05-04:00 1992-07-31T05:48:05-04:00


In [9]:
for entry in bundle['entry']:
    resource = entry['resource']
    resource_type = resource['resourceType']
    if resource_type == 'Claim':
        id = resource['id']
        patient = resource['patient']['reference']
        status = resource['status']
        total = resource['total']['value']
        currency = resource['total']['currency']
        start = parse(resource['billablePeriod']['start'])
        end = parse(resource['billablePeriod']['end'])
        print(patient, status, total, currency, start, end)
        break

urn:uuid:07b3c863-ea51-45a7-b446-493b39eb72e2 active 125.0 USD 1992-07-31 05:18:05-04:00 1992-07-31 05:48:05-04:00


In [10]:
import sqlalchemy as db

In [11]:
engine = db.create_engine('sqlite:///fhir.sqlite')
connection = engine.connect()
metadata = db.MetaData()

In [12]:
claims = db.Table('claims', metadata,
                  db.Column('id', db.Integer()),
                  db.Column('patient', db.String(255)),
                  db.Column('status', db.String(255)),
                  db.Column('total', db.Float()),
                  db.Column('currency', db.String(255)),
                  db.Column('start', db.DateTime()),
                  db.Column('end', db.DateTime()))

metadata.create_all(engine) #Creates the table

In [13]:
print(claims.columns.keys())

['id', 'patient', 'status', 'total', 'currency', 'start', 'end']


In [14]:
for entry in bundle['entry']:
    resource = entry['resource']
    resource_type = resource['resourceType']
    if resource_type == 'Claim':
        id = resource['id']
        patient = resource['patient']['reference']
        status = resource['status']
        total = resource['total']['value']
        currency = resource['total']['currency']
        start = parse(resource['billablePeriod']['start'])
        end = parse(resource['billablePeriod']['end'])        
        query = db.insert(claims).values(id=id, 
                                         patient=patient,
                                         status=status,
                                         total=total,
                                         currency=currency,
                                         start=start,
                                         end=end)

        result_proxy = connection.execute(query)

In [15]:
stmt = db.select([claims])
results = connection.execute(stmt).fetchall()

In [16]:
import pandas as pd

In [17]:
df = pd.DataFrame(results)

In [18]:
df.columns = results[0].keys()

In [19]:
df.head()

,id,patient,status,total,currency,start,end
0,b16bf2ad-9084-4764-b029-83aa5e40edaa,urn:uuid:ab8f33ad-54b4-4ee0-acd7-a14918459c3d,active,255.0,USD,1988-04-23 13:35:04,1988-04-23 13:50:04
1,b16bf2ad-9084-4764-b029-83aa5e40edaa,urn:uuid:ab8f33ad-54b4-4ee0-acd7-a14918459c3d,active,255.0,USD,1988-04-23 13:35:04,1988-04-23 13:50:04
2,722b5412-303f-4c19-8065-b40c8c3cff26,urn:uuid:ab8f33ad-54b4-4ee0-acd7-a14918459c3d,active,125.0,USD,1988-04-23 13:35:04,1988-04-23 13:50:04
3,6305265e-1ffb-48af-bc29-404eafe9e426,urn:uuid:ab8f33ad-54b4-4ee0-acd7-a14918459c3d,active,125.0,USD,1991-08-03 13:35:04,1991-08-03 13:50:04
4,e6f0f9cb-0655-4e3f-bbb3-b11fe034618a,urn:uuid:ab8f33ad-54b4-4ee0-acd7-a14918459c3d,active,125.0,USD,1997-08-09 13:35:04,1997-08-09 14:50:04


In [20]:
stmt = db.select([claims])
results = connection.execute(stmt).fetchall()
pd.DataFrame(results).head()
df.columns = results[0].keys()
df.head()

,id,patient,status,total,currency,start,end
0,b16bf2ad-9084-4764-b029-83aa5e40edaa,urn:uuid:ab8f33ad-54b4-4ee0-acd7-a14918459c3d,active,255.0,USD,1988-04-23 13:35:04,1988-04-23 13:50:04
1,b16bf2ad-9084-4764-b029-83aa5e40edaa,urn:uuid:ab8f33ad-54b4-4ee0-acd7-a14918459c3d,active,255.0,USD,1988-04-23 13:35:04,1988-04-23 13:50:04
2,722b5412-303f-4c19-8065-b40c8c3cff26,urn:uuid:ab8f33ad-54b4-4ee0-acd7-a14918459c3d,active,125.0,USD,1988-04-23 13:35:04,1988-04-23 13:50:04
3,6305265e-1ffb-48af-bc29-404eafe9e426,urn:uuid:ab8f33ad-54b4-4ee0-acd7-a14918459c3d,active,125.0,USD,1991-08-03 13:35:04,1991-08-03 13:50:04
4,e6f0f9cb-0655-4e3f-bbb3-b11fe034618a,urn:uuid:ab8f33ad-54b4-4ee0-acd7-a14918459c3d,active,125.0,USD,1997-08-09 13:35:04,1997-08-09 14:50:04


In [21]:
stmt = db.select([claims]).where(claims.columns.total >= 25000.0).order_by(db.desc(claims.columns.total))
results = connection.execute(stmt).fetchall()
df = pd.DataFrame(results).head()
df.columns = results[0].keys()
df.head()

,id,patient,status,total,currency,start,end
0,16f8da2a-322e-47ca-bc82-8b66843858f9,urn:uuid:ab8f33ad-54b4-4ee0-acd7-a14918459c3d,active,74670.12,USD,2015-12-26 12:35:04,2015-12-26 17:50:04
1,16f8da2a-322e-47ca-bc82-8b66843858f9,urn:uuid:ab8f33ad-54b4-4ee0-acd7-a14918459c3d,active,74670.12,USD,2015-12-26 12:35:04,2015-12-26 17:50:04
2,16f8da2a-322e-47ca-bc82-8b66843858f9,urn:uuid:ab8f33ad-54b4-4ee0-acd7-a14918459c3d,active,74670.12,USD,2015-12-26 12:35:04,2015-12-26 17:50:04
3,16f8da2a-322e-47ca-bc82-8b66843858f9,urn:uuid:ab8f33ad-54b4-4ee0-acd7-a14918459c3d,active,74670.12,USD,2015-12-26 12:35:04,2015-12-26 17:50:04


In [22]:
stmt = db.select([db.func.sum(claims.columns.total).label('total claims'),
                  db.func.avg(claims.columns.total).label('average claim')])
results = connection.execute(stmt).fetchall()
df = pd.DataFrame(results).head()
df.columns = results[0].keys()
df.head()

,total claims,average claim
0,1413740.91,4336.628558
